In [1]:
import os
from bs4 import BeautifulSoup
import urllib.request
import datetime
import pandas as pd
import altair as alt
from Functions import *
from MatchClass import *

In [2]:
def convert_date_season(date):
    '''
    This functions receives a date in format "weekday, month day, year" and returns it in format "YYYYMMDD"
    '''
    date = datetime.datetime.strptime(date, "%a, %b %d, %Y")
    return date.strftime("%Y/%m/%d")


def treat_drought(drought):
    hours, mins, seconds = drought.split(":")
    return int(mins) + int(seconds)/60


def get_value(game, statistic):
    if statistic == "streak":
        return game.greatest_streak()
    if statistic == "partial":
        return game.greatest_partial()
    if statistic == "drought":
        value = game.longest_drought()
        return list(map(treat_drought, value))
        

In [29]:
def treat_match(row, team, currentDate, table, statistic):
    '''
    This function treats all the matches in a season and does the desired computation for each of them
    - row: row in the season table (bs4.element.Tag)
    - team: name of the team. It can be the city, the club name or a combination (string)
    - currentDate: date at the time of the query (string)
    '''
    cols = row.find_all('td')
    if len(cols) == 14:
        date = cols[0].text
        location = cols[4].text
        opTeam = cols[5].text

        date = convert_date_season(date)
        isAway = (location == "@")
        if isAway:
            home = opTeam
            away = team
        else:
            home = team
            away = opTeam
        if date < currentDate:
            game = Match(home, away, date)
            value = get_value(game, statistic)
            row = [date, opTeam, value[isAway]]
            row = pd.Series(row, index=["Date", "Opponent", "Value"])
            table = table.append(row, ignore_index=True)
    return table


def plot_line(table, statistic):
    chart = alt.Chart(
        table.reset_index()
    ).mark_line(
        point=True
    ).encode(
        x = alt.X('index:T', title = "match"),
        y = alt.Y('Value:Q', title = statistic),
        tooltip = ['Date:T', 'Opponent:N']
    ).add_selection(
        alt.selection_single()
    )

    rule = alt.Chart(table).mark_rule(color='darkblue').encode(
        y = alt.Y('mean(Value):Q')
    )
    return chart + rule

In [30]:
def main(team, season, statistic):
    '''
    This function explores all the matches of a team during a season
    - team: name of the team. It can be the city, the club name or a combination (string)
    - season: season that is going to be analyzed (string)
    '''
    os.chdir(os.path.abspath(''))

    currentDate = datetime.date.today().strftime("%Y/%m/%d")
    shortTeam = get_team(team)
    season = season.split("-")[1]
    webpage = f"https://www.basketball-reference.com/teams/{shortTeam}/{season}_games.html"

    response = urllib.request.urlopen(webpage)
    htmlDoc = response.read()
    soup = BeautifulSoup(htmlDoc, 'html.parser')
    res = soup.find_all('tr')

    table = pd.DataFrame(columns=["Date", "Opponent", "Value"])
    
    for line in res:
        table = treat_match(line, team, currentDate, table, statistic)

    return plot_line(table, statistic)

In [31]:
graf = main("Memphis", "2020-2021", "streak")
graf.properties(width=750)

alt.LayerChart(...)

In [32]:
graf = main("Memphis", "2020-2021", "partial")
graf.properties(width=750)

alt.LayerChart(...)

In [33]:
graf = main("Memphis", "2020-2021", "drought")
graf.properties(width=750)

alt.LayerChart(...)